# Generating new text with HuggingFace Transformers

In this example, we'll use pretrained [HuggingFace](https://github.com/huggingface/transformers) instance of TransformerXL to generate new text.

Let's start with the imports:

In [1]:
import torch
from transformers import TransfoXLLMHeadModel, TransfoXLTokenizer

I1209 05:04:09.205170 139652933334848 file_utils.py:32] TensorFlow version 2.0.0 available.
I1209 05:04:09.206719 139652933334848 file_utils.py:39] PyTorch version 1.3.1 available.
I1209 05:04:09.548416 139652933334848 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


Let's obtain the device (try for GPU, otherwise use CPU):

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Next, we'll instantiate pre-trained model-specific tokenizer and the model itself:

In [3]:
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl-wt103')
model = TransfoXLLMHeadModel.from_pretrained('transfo-xl-wt103').to(device)

I1209 05:06:01.744601 139652933334848 tokenization_utils.py:374] loading file https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-vocab.bin from cache at /home/hok/.cache/torch/transformers/b24cb708726fd43cbf1a382da9ed3908263e4fb8a156f9e0a4f45b7540c69caa.a6a9c41b856e5c31c9f125dd6a7ed4b833fbcefda148b627871d4171b25cffd1
I1209 05:06:02.524099 139652933334848 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/transfo-xl-wt103-config.json from cache at /home/hok/.cache/torch/transformers/a6dfd6a3896b3ae4c1a3c5f26ff1f1827c26c15b679de9212a04060eaf1237df.aef76fb1064c932cd6a2a2be3f23ebbfa5f9b6e29e8e87b571c45b4a5d5d1b90
I1209 05:06:02.528438 139652933334848 configuration_utils.py:168] Model config {
  "adaptive": true,
  "attn_type": 0,
  "clamp_len": 1000,
  "cutoffs": [
    20000,
    40000,
    200000
  ],
  "d_embed": 1024,
  "d_head": 64,
  "d_inner": 4096,
  "d_model": 1024,
  "div_val": 4,
  "dropatt": 0.0,
  "dropou

Next, we'll create an initial text sequence, which will serve as a seed. The model will generate new content based on that sequence:

In [4]:
# Initial input sequence
text = "The company was founded in"
tokens_tensor = \
    torch.tensor(tokenizer.encode(text)) \
        .unsqueeze(0) \
        .to(device)

Finally, we'll use the `model` to generating new word tokens step-by-step. The process will continue until END-OF-SEQUENCE token is reached:

In [5]:
mems = None  # recurrence mechanism

predicted_tokens = list()
for i in range(50):  # stop at 50 predicted tokens
    # Generate predictions
    predictions, mems = model(tokens_tensor, mems=mems)

    # Get most probable word index
    predicted_index = torch.topk(predictions[0, -1, :], 1)[1]

    # Extract the word from the index
    predicted_token = tokenizer.decode(predicted_index)

    # break if [EOS] reached
    if predicted_token == tokenizer.eos_token:
        break

    # Store the current token
    predicted_tokens.append(predicted_token)

    # Append new token to the existing sequence
    tokens_tensor = torch.cat((tokens_tensor, predicted_index.unsqueeze(1)), dim=1)

print('Initial sequence: ' + text)
print('Predicted output: ' + " ".join(predicted_tokens))

Initial sequence: The company was founded in
Predicted output: the United States .


As we can see, the generated text is relevant to the initial sequence.